In [129]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

In [37]:
api_key = "508b0294291fe31d619688ac095b3d0b"
base_url = "https://financialmodelingprep.com/api/v3/"


In [63]:
#getting symbols that are tradable and also have financial statements

statement_symbols_list = requests.get(f"{base_url}financial-statement-symbol-lists?apikey={api_key}").json

tradable_symbols_list = requests.get(f"{base_url}available-traded/list?apikey={api_key}")
tradable_symbols_list = tradable_symbols_list.json()
symbols_tradable = [obj['symbol'] for obj in tradable_symbols_list]

symbol_market_cap = pd.read_csv("by_market_cap.csv")
symbol_market_cap['has_statement'] = False
symbol_market_cap['is_tradable'] = False

for i in range(len(symbol_market_cap)):
     if symbol_market_cap['Symbol'][i] in symbols_tradable:
          symbol_market_cap['is_tradable'][i] = True
     if symbol_market_cap['Symbol'][i] in statement_symbols_list:
          symbol_market_cap['has_statement'][i] = True

target_symbols = symbol_market_cap[(symbol_market_cap['has_statement'] == True) & (symbol_market_cap['is_tradable']== True)].head(100)

C:\Users\Asus\AppData\Local\Temp\ipykernel_1536\3030495244.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  symbol_market_cap['is_tradable'][i] = True
C:\Users\Asus\AppData\Local\Temp\ipykernel_1536\3030495244.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  symbol_market_cap['has_statement'][i] = True


In [85]:
#getting the financial analysis for each symbol
def Statement_analysis_caller(symbol, years = 10):
     response = requests.get(f"{base_url}key-metrics/{symbol}?limit={years}&apikey={api_key}").json()
     return response

In [107]:
#gathering historical data in one data frame
df = pd.DataFrame()
for s in target_symbols['Symbol']:
     rows = Statement_analysis_caller(s, years = 6)
     try:
          # Check if rows is not empty before accessing elements
          if rows:
               for i in range(len(rows)):
                    df = df.append(rows[i], ignore_index=True)
                    
                    # Introduce a 2-second delay
               time.sleep(2)
     except KeyError as e:
          print(f"Error in processing {s}: {e}")
          continue  # Continue to the next iteration if an error occurs



Error in processing MC.PA: 0
Error in processing 005930.KS: 0
Error in processing 600519.SS: 0
Error in processing NESN.SW: 0
Error in processing OR.PA: 0
Error in processing 1398.HK: 0
Error in processing ROG.SW: 0
Error in processing RMS.PA: 0
Error in processing RELIANCE.NS: 0
Error in processing 0941.HK: 0
Error in processing 601857.SS: 0
Error in processing 601288.SS: 0
Error in processing TCS.NS: 0
Error in processing 601988.SS: 0
Error in processing 601939.SS: 0
Error in processing PRX.AS: 0
Error in processing CDI.PA: 0


In [108]:
df

,symbol,date,calendarYear,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,AAPL,2023-09-30,2023,FY,24.344473,6.160669,7.021175,6.325110,3.909686,3.947224,...,63363000000,5638500000,58.075649,106.721468,10.791292,6.284906,3.420118,33.823567,1.560760,-0.696064
1,AAPL,2022-09-24,2022,FY,24.317273,6.154614,7.532763,6.872426,2.978793,3.124822,...,59439000000,5763000000,56.400205,104.685277,8.075698,6.471608,3.486641,45.197331,1.969589,-0.660337
2,AAPL,2021-09-25,2021,FY,21.903541,5.669029,6.229346,5.565624,3.750553,3.777557,...,48529500000,5320500000,51.390969,93.851071,11.276593,7.102415,3.889140,32.367933,1.500713,-0.663722
3,AAPL,2020-09-26,2020,FY,15.820258,3.308587,4.649230,4.228014,5.241031,3.765477,...,44266000000,4083500000,49.787534,91.048190,8.741883,7.331152,4.008866,41.753016,0.878664,-0.421217
4,AAPL,2019-09-28,2019,FY,14.085283,2.991446,3.756685,3.188508,5.443948,4.898834,...,51062000000,4031000000,64.258765,104.314077,9.263639,5.680159,3.499048,39.401364,0.610645,-0.568178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,RY,2022-10-31,2022,FY,34.748592,11.252061,15.632057,13.850992,171.406914,76.987634,...,13895500000,0,196.355407,0.000000,0.000000,1.858874,0.000000,0.000000,0.146154,-1.781066
411,RY,2021-10-31,2021,FY,34.778842,11.259928,42.857654,41.322912,162.197589,69.271938,...,9984500000,0,166.028322,0.000000,0.000000,2.198420,0.000000,0.000000,0.162547,-1.534743
412,RY,2020-10-31,2020,FY,33.080626,8.028569,97.491072,95.644754,209.032140,60.863184,...,8899000000,0,144.360352,0.000000,0.000000,2.528395,0.000000,0.000000,0.131912,-1.846318
413,RY,2019-10-31,2019,FY,32.009111,8.963053,9.942298,8.366445,116.481354,58.213146,...,8083500000,0,156.813461,0.000000,0.000000,2.327606,0.000000,0.000000,0.153970,-1.575852


In [134]:
#getting historical price data for a symbol
def Get_historical_data (symbol, start_date, end_date):
     response = requests.get(f"{base_url}historical-price-full/{symbol}?from={start_date}&to={end_date}&apikey={api_key}").json()
     response = response['historical']
     return response



In [135]:

price_data = pd.DataFrame()
start_date = datetime.strptime("2018-6-1", "%Y-%m-%d").date()
end_date = datetime.strptime("2023-2-1", "%Y-%m-%d").date()
symbols = df['symbol'].unique()
for s in symbols:
     data = Get_historical_data(s, start_date, end_date)
     for i in range(len(data)):
          new_row = data[i]
          new_row['symbol'] = s
          price_data = price_data.append(new_row, ignore_index=True)
          

In [136]:
price_data

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2023-02-01,143.97,146.61,141.32,145.43,144.62,77663600,77663600,1.46,1.01000,144.45,"February 01, 23",0.010100,AAPL
1,2023-01-31,142.70,144.34,142.28,144.29,143.49,65874500,65874500,1.59,1.11000,143.64,"January 31, 23",0.011100,AAPL
2,2023-01-30,144.96,145.55,142.85,143.00,142.21,64015300,64015300,-1.96,-1.35000,143.80,"January 30, 23",-0.013500,AAPL
3,2023-01-27,143.16,147.23,143.08,145.93,145.12,70555800,70555800,2.77,1.93000,145.41,"January 27, 23",0.019300,AAPL
4,2023-01-26,143.17,144.25,141.90,143.96,143.16,54105100,54105100,0.79,0.55179,143.37,"January 26, 23",0.005518,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97177,2018-06-07,76.81,76.94,76.25,76.53,62.04,494200,494200,-0.28,-0.36454,76.57,"June 07, 18",-0.003645,RY
97178,2018-06-06,77.15,77.18,76.51,76.61,62.10,573800,573800,-0.54,-0.69994,76.77,"June 06, 18",-0.006999,RY
97179,2018-06-05,76.25,76.61,75.98,76.50,62.02,696200,696200,0.25,0.32787,76.36,"June 05, 18",0.003279,RY
97180,2018-06-04,75.96,76.84,75.96,76.49,62.01,682900,682900,0.53,0.69774,76.43,"June 04, 18",0.006977,RY


In [137]:
# converting data frames to csv files
df.to_csv("financial_parameters.csv")
price_data.to_csv("price_data.csv")